In [1]:
import os
import pandas as pd
from pandas import DataFrame
from datetime import datetime as dt
import matplotlib.pyplot as plt

# Imports the method used for connecting to DBs
from sqlalchemy import create_engine, inspect

# Allow us to declare column types
from sqlalchemy import Column, Integer, String, Float, Boolean 

from sqlalchemy.orm import Session

In [2]:
engine = create_engine("sqlite:///../Data/megawatt.sqlite")

In [3]:
newdf = DataFrame(engine.execute("SELECT * FROM demand where date >'2016-01-01'and date < '2018-12-01' and name='ISNE'").fetchall())
newdf = newdf.rename(columns={
    newdf.columns[0]: 'datetime',
    newdf.columns[1]: 'demand',
    newdf.columns[2]: 'abbr',
    newdf.columns[3] : 'datetime_utc',
    newdf.columns[4] : 'date',
    newdf.columns[5] : 'hour'
    })

newdf['datetime'] = pd.to_datetime(newdf['datetime'])
newdf['datetime_utc'] = pd.to_datetime(newdf['datetime_utc'])
# newdf.sort_values('datetime',ascending=True)
newdf.head()


,datetime,demand,abbr,datetime_utc,date,hour
0,2018-10-20 20:00:00,12597.0,ISNE,2018-10-21 01:00:00,2018-10-20,20
1,2018-10-20 19:00:00,13047.0,ISNE,2018-10-21 00:00:00,2018-10-20,19
2,2018-10-20 18:00:00,13205.0,ISNE,2018-10-20 23:00:00,2018-10-20,18
3,2018-10-20 17:00:00,12563.0,ISNE,2018-10-20 22:00:00,2018-10-20,17
4,2018-10-20 16:00:00,12161.0,ISNE,2018-10-20 21:00:00,2018-10-20,16


In [4]:
weather_file = os.path.join('../Data','2016-2018_MA_weather_data.csv')

In [5]:
raw_listings_df = pd.read_csv(weather_file, encoding="ISO-8859-1")

In [6]:
raw_listings_df['datetime'] = pd.to_datetime(raw_listings_df['datetime'])
raw_listings_df = raw_listings_df.rename(columns={
    'datetime':'datetime_utc'
})
raw_listings_df.head()

,Unnamed: 0,usaf,wban,windir,windspeed,skycond,tempc,dewpoint,tempf,datetime_utc
0,0,725090,14739,270,57.0,99999,5.6,28.04,42.08,2016-01-01 00:00:00
1,1,725090,14739,260,51.0,22000,5.0,28.04,41.00,2016-01-01 01:00:00
2,2,725090,14739,260,62.0,1433,5.0,26.96,41.00,2016-01-01 02:00:00
3,3,725090,14739,260,46.0,1433,4.4,26.96,39.92,2016-01-01 03:00:00
4,5,725090,14739,260,57.0,1311,4.4,26.96,39.92,2016-01-01 04:00:00


In [7]:
del raw_listings_df['Unnamed: 0']

In [8]:
raw_listings_df.head()

,usaf,wban,windir,windspeed,skycond,tempc,dewpoint,tempf,datetime_utc
0,725090,14739,270,57.0,99999,5.6,28.04,42.08,2016-01-01 00:00:00
1,725090,14739,260,51.0,22000,5.0,28.04,41.00,2016-01-01 01:00:00
2,725090,14739,260,62.0,1433,5.0,26.96,41.00,2016-01-01 02:00:00
3,725090,14739,260,46.0,1433,4.4,26.96,39.92,2016-01-01 03:00:00
4,725090,14739,260,57.0,1311,4.4,26.96,39.92,2016-01-01 04:00:00


In [9]:
whatdf = pd.merge(newdf, raw_listings_df, on='datetime_utc',how='inner')
whatdf.head()

,datetime,demand,abbr,datetime_utc,date,hour,usaf,wban,windir,windspeed,skycond,tempc,dewpoint,tempf
0,2018-10-14 00:00:00,10443.0,ISNE,2018-10-14 05:00:00,2018-10-14,0,725090,14739,999,NaN,99999,NaN,NaN,NaN
1,2018-10-13 23:00:00,10884.0,ISNE,2018-10-14 04:00:00,2018-10-13,23,725090,14739,310,31.0,22000,8.3,39.92,46.94
2,2018-10-13 22:00:00,11436.0,ISNE,2018-10-14 03:00:00,2018-10-13,22,725090,14739,290,41.0,22000,9.4,39.92,48.92
3,2018-10-13 21:00:00,12203.0,ISNE,2018-10-14 02:00:00,2018-10-13,21,725090,14739,280,46.0,22000,10.6,39.92,51.08
4,2018-10-13 20:00:00,12774.0,ISNE,2018-10-14 01:00:00,2018-10-13,20,725090,14739,280,31.0,22000,10.6,39.92,51.08


In [10]:
whatdf.to_csv('mergedSample.csv')